In [1]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score
from openai import OpenAI

In [ ]:
corpus = pd.read_json("corpus.jsonl", lines=True)
queries = pd.read_json("queries.jsonl", lines=True)
corpus.set_index("_id", inplace=True)
queries.set_index("_id", inplace=True)
qrels = pd.read_csv("top_qrels.tsv", sep="\t")
qrels

,query-id,corpus-id,score
0,bwq-0,bwc-9,2
1,bwq-1,bwc-73,1
2,bwq-1,bwc-70,2
3,bwq-1,bwc-78,2
4,bwq-2,bwc-653,2
...,...,...,...
1550,bwq-534,bwc-166896,2
1551,bwq-534,bwc-166918,1
1552,bwq-534,bwc-166919,2
1553,bwq-534,bwc-166923,1


In [3]:
class DeepSeekJudge:
  def __init__(self):
    self.client = OpenAI(api_key="sk-21cb3976392646d687940daed0ca3650", base_url="https://api.deepseek.com")
  def generate_judgement(self, query, passage):
    system_message = """
Ты — строгий эксперт по проверке фактов и оценке семантической релевантности. Твоя задача — проанализировать запрос и отрывок текста, чтобы определить, насколько отрывок подтверждает или содержит информацию из запроса.

Используй следующую шкалу для оценки (целое число от 0 до 2):

[2] Полностью релевантный: Отрывок содержит исчерпывающую информацию, подтверждающую весь факт из запроса. Смысл запроса полностью раскрыт в тексте.
[1] Частично релевантный: Отрывок касается темы запроса, упоминает ключевые сущности или содержит часть факта, но не дает полного подтверждения или ответа.
[0] Нерелевантный: Отрывок не имеет отношения к запросу, не содержит упоминаний ключевых объектов или противоречит контексту.

ВАЖНО:
1. Твой ответ должен быть СТРОГО в формате JSON.
2. Используй ключ "score".
3. Значение должно быть только числом: 0, 1 или 2.
4. Не пиши никаких пояснений, комментариев или вводных слов. Только JSON.

Пример формата ответа:
{"score": 1}
"""
    user_message_template = f"""
Оцени релевантность следующей пары текстов, используя заданные критерии.

ЗАПРОС:
{query}

ОТРЫВОК:
{passage}

Выведи оценку в JSON формате: {{"score": N}}
"""
    response = self.client.chat.completions.create(
      model="deepseek-chat",
      messages=[
          {"role": "system", "content": system_message},
          {"role": "user", "content": user_message_template},
      ],
      stream=False,
      response_format={
        'type': 'json_object'
      }
    )
    return response.choices[0].message.content

In [4]:
judge = DeepSeekJudge()

In [5]:
results = []
test = qrels.head(100)
for idx, row in tqdm(test.iterrows(), total=len(test)):
  true_score = row["score"]
  query = queries.loc[row["query-id"]]["text"]
  response = corpus.loc[row["corpus-id"]]["text"]
  res = judge.generate_judgement(query, response)
  res = json.loads(res)
  result = {
    "llm-score": res["score"],
    "human-score": true_score,
    "query-id": row["query-id"],
    "corpus-id": row["corpus-id"]

  }
  results.append(result)

100%|██████████| 100/100 [02:45<00:00,  1.65s/it]


In [6]:
results_df = pd.DataFrame(results)
from sklearn.metrics import cohen_kappa_score
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
cohen_kappa_score(llm, human)

0.039548022598870025

In [7]:
import numpy as np
import metrics


print("NDCG: ", metrics.calc_ndcg(results_df))
print("NRMSE: ", metrics.calc_nrmse(results_df))
print("NMAE: ", metrics.calc_nmae(results_df))
print("kendalltau: ", metrics.calc_kendalltau(results_df))
print("RBO: ", metrics.calc_rbo(results_df))

NDCG:  0.9861071634012046
NRMSE:  0.47958315233127197
NMAE:  0.38
kendalltau:  0.5329382517914113
RBO:  0.2570461955999999


In [8]:
pd.DataFrame(results_df).to_csv('sents_llm_deepseek_gemini_fact.tsv', sep='\t', index=False)

In [9]:
from sklearn.metrics import confusion_matrix
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
confusion_matrix(human, llm, labels=range(3))

array([[ 0,  0,  0],
       [30, 10,  0],
       [ 8, 30, 22]])

In [10]:
res = results_df[(results_df['llm-score'] == 1) & (results_df['human-score'] == 2)]

for _, row in res.iterrows():
  print("LLM: ", row["llm-score"], "HUMAN: ", row["human-score"], "ID: ", row["query-id"])
  print("Query: ", queries.loc[row["query-id"]]["text"])
  print("Passage: ", corpus.loc[row["corpus-id"]]["text"])

LLM:  1 HUMAN:  2 ID:  bwq-1
Query:  Обидевшись, старший брат записал дисс на младшего .
Passage:  Оригинальная версия трека содержала критику, направленную в адрес младшего брата Майкла Джексона.
LLM:  1 HUMAN:  2 ID:  bwq-2
Query:  « Ваня » , пройдясь по Британии , обрушился ледяным дождём на юг России .
Passage:  В России влияние циклона больше всего ощутили на юге европейской части.
LLM:  1 HUMAN:  2 ID:  bwq-2
Query:  « Ваня » , пройдясь по Британии , обрушился ледяным дождём на юг России .
Passage:  В Ростове-на-Дону 12 декабря из-за ледяного дождя, который, с небольшими перерывами, шёл пять дней, остановили движение общественного транспорта, а 13 декабря в школах отменили занятия.
LLM:  1 HUMAN:  2 ID:  bwq-3
Query:  Охота на « врагов народа », начавшаяся после убийства Кирова , не делала скидок на возраст преследуемых .
Passage:  После его убийства был введён упрощённый порядок ведения следствия; смертные приговоры приводились в исполнения в течение 24-х часов.
LLM:  1 HUMAN:  